<a href="https://colab.research.google.com/github/dakilaledesma/herbarium_model_testing/blob/main/2x3xnet/H2022_GPU_DualNet_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! unzip -q /content/drive/MyDrive/UNC/H2022/RCNN_PP/pls224p56_Orchidaceae_test.zip -d /content/
! unzip -q /content/drive/MyDrive/UNC/H2022/orchidaceae_test.zip -d /content/

In [8]:
! pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.1 MB/s 


In [9]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.preprocessing import image
from tqdm.notebook import tqdm
import numpy as np
from tensorflow.keras.models import Model, load_model
import tensorflow_addons as tfa

In [10]:
model = load_model("/content/drive/MyDrive/UNC/H2022/DualNet2_adabelief", custom_objects={"Addons>AdaBelief": tfa.optimizers.AdaBelief(learning_rate=1e-3)})

In [11]:
from glob import glob
import os
cat_to_int = {}
for i, f in enumerate(sorted(glob("pls224p56_Orchidaceae_test/*"))):
  cat = os.path.basename(f)
  cat_to_int[cat] = i

In [12]:
print(cat_to_int.items())

dict_items([('00098', 0), ('00099', 1), ('00100', 2), ('00915', 3), ('01032', 4), ('01306', 5), ('01819', 6), ('01854', 7), ('01963', 8), ('01964', 9), ('02127', 10), ('02133', 11), ('02134', 12), ('02135', 13), ('02136', 14), ('02137', 15), ('02138', 16), ('02246', 17), ('02247', 18), ('02394', 19), ('02395', 20), ('02396', 21), ('02397', 22), ('02398', 23), ('02417', 24), ('02470', 25), ('02471', 26), ('02472', 27), ('02473', 28), ('02474', 29), ('03139', 30), ('03140', 31), ('03141', 32), ('03142', 33), ('03143', 34), ('03144', 35), ('03145', 36), ('03155', 37), ('03156', 38), ('03157', 39), ('03158', 40), ('03159', 41), ('03160', 42), ('03161', 43), ('03162', 44), ('03163', 45), ('03258', 46), ('03658', 47), ('03766', 48), ('03775', 49), ('03776', 50), ('03852', 51), ('03889', 52), ('03890', 53), ('03891', 54), ('03892', 55), ('03893', 56), ('03894', 57), ('03984', 58), ('03994', 59), ('04019', 60), ('04020', 61), ('04326', 62), ('04327', 63), ('04329', 64), ('04490', 65), ('04491'

In [13]:
valid_fn_t = {}
for fn in sorted(glob("pls224p56_Orchidaceae_test/**/*.*", recursive=True)):
  bn = os.path.basename(fn)
  valid_fn_t[bn] = fn

o_t = []
pl_t = []
y_t = []
orc_t = sorted(glob("orchidaceae_test/**/*.*", recursive=True))
for fn in tqdm(orc_t, total=len(list(orc_t))):
  try:
    bn =  os.path.basename(fn)
    cat = cat_to_int[fn.split("/")[-2]]
    try:
      pl_fn = valid_fn_t[bn]
    except KeyError:
      pass

    img = image.load_img(fn, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    o_t.append(x)

    img = image.load_img(pl_fn, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    pl_t.append(x)

    y_t.append(cat)
  except:
    pass


  0%|          | 0/4577 [00:00<?, ?it/s]

In [14]:
preds = []
for ot, plt in tqdm(zip(o_t, pl_t), total=len(o_t)):
  ot = np.array(ot).reshape((-1, 224, 224, 3))
  plt = np.array(plt).reshape((-1, 224, 224, 3))
  pred = np.argmax(model([ot, plt])[0])
  preds.append(pred)

  0%|          | 0/4577 [00:00<?, ?it/s]

In [15]:
from sklearn.metrics import accuracy_score

print(accuracy_score(y_t, preds))

0.3792877430631418


In [16]:
for x, y in zip(preds, y_t):
  print(x, y)

223 0
223 0
146 0
164 1
165 1
165 1
165 1
165 1
164 1
163 1
164 1
171 2
107 2
107 2
175 2
135 2
96 2
3 3
3 3
52 3
3 3
127 3
3 3
3 3
3 3
3 3
3 3
100 3
100 3
164 3
33 3
33 3
3 3
57 3
23 4
205 4
55 4
205 4
205 4
24 4
3 4
296 4
56 4
56 4
4 4
21 4
24 4
4 4
4 4
57 4
4 4
255 4
55 4
208 4
4 4
4 4
57 4
5 5
5 5
5 5
5 5
165 5
165 5
5 5
215 5
165 5
5 5
135 5
165 5
219 5
219 5
219 5
219 5
5 5
99 5
5 5
5 5
165 5
91 5
121 5
163 5
5 5
163 6
163 6
163 6
3 6
94 6
278 7
178 7
178 7
7 7
178 7
7 7
178 7
178 7
165 7
222 7
163 7
155 7
146 7
222 7
178 7
178 7
231 8
113 8
112 8
231 8
8 8
113 8
8 8
8 8
8 8
8 8
114 8
90 8
91 8
85 8
160 9
9 9
9 9
9 9
9 9
114 9
57 9
114 9
9 9
104 9
9 9
113 9
248 9
8 9
20 9
9 9
248 9
257 9
250 9
37 9
10 10
178 10
10 10
275 10
178 10
178 10
165 10
10 10
255 10
255 10
100 11
100 11
282 11
108 11
255 11
223 11
187 11
104 11
165 12
30 12
239 12
14 13
16 13
160 14
76 14
16 14
16 14
9 14
165 14
9 14
154 14
165 14
9 14
231 14
107 14
154 14
233 14
41 14
41 14
49 14
129 14
129 15
233 15
9 1